In [1]:
from dateutil.relativedelta import relativedelta
from coinbase.wallet.client import Client
from datetime import datetime, timedelta
import robin_stocks.robinhood as rh
import pandas as pd
import PyPDF2
import json
import time
import os
import re

In [2]:
# Creds... 
from retrieve_creds import retrieve_creds_for_money_manager
creds = retrieve_creds_for_money_manager()
robinhood_u = creds['Robinhood'][0]
robinhood_p = creds['Robinhood'][1]
coinbase_key_id = creds['Coinbase'][0]
coinbase_secret = creds['Coinbase'][1]

In [3]:
# Login
login = rh.authentication.login(robinhood_u, robinhood_p, mfa_code = "")
client = Client(coinbase_key_id, coinbase_secret)

In [4]:
# +++ Robinhood +++

# would login here.. 

# Get all stock orders (api call, filter out cancelled orders, and subset cols)
all_stock_orders = pd.DataFrame(
    pd.json_normalize( rh.orders.get_all_stock_orders() )
)
all_stock_orders = all_stock_orders[ all_stock_orders['state']!="cancelled" ]
all_stock_orders = all_stock_orders[ all_stock_orders['created_at']!='2021-11-29T14:38:51.740313Z' ] # Filter out this sell order, as well.
all_stock_orders = all_stock_orders[
    ["instrument","created_at","side","quantity","executed_notional.amount"]
]

# Set numeric data types
all_stock_orders["executed_notional.amount"] = all_stock_orders["executed_notional.amount"].astype("float64")
all_stock_orders["quantity"] = all_stock_orders["quantity"].astype("float64")

# Grab more data about each instrument (more details, fundamentals, and such) 
# Then log out of Robinhood
instrument_details = []
for instrument_url in all_stock_orders["instrument"].unique():
    
    # Retreive instrument data
    instrument_json_resp = rh.stocks.get_instrument_by_url(instrument_url)
    symbol = instrument_json_resp["symbol"]
    simple_name = instrument_json_resp["simple_name"]
    type = instrument_json_resp["type"]

    fundamentals_json_resp = rh.stocks.get_fundamentals(instrument_json_resp["symbol"])[0]
    if fundamentals_json_resp:
        
        sector = fundamentals_json_resp["sector"]
        industry = fundamentals_json_resp["industry"]
        latest_price = float( rh.stocks.get_latest_price(instrument_json_resp["symbol"])[0] )

    else:
        sector = ""
        industry = "" 
        latest_price = 0.0
        simple_name = ""

    # Append new record of instrument details
    instrument_details.append(
        (instrument_url, symbol, simple_name, type, sector, industry, latest_price)
    )

# Convert list of instrument details into a Pandas df
instrument_details = pd.DataFrame(
    instrument_details, 
    columns=[
        "instrument",
        "symbol",
        "simple_name",
        "investment type",
        "sector",
        "industry",
        "latest_price"
    ] # df col names...
)

# Log out of Robinhood, we're done using its web APIs. 
# rh.authentication.logout() leave this alone for now... 

# Clean up
del instrument_json_resp, fundamentals_json_resp
del login

# Join all stock orders with instrument data 
all_stock_orders = pd.merge(all_stock_orders, instrument_details, on = "instrument")
del instrument_details

# rename cols so that we can apply the same code to Robinhood and coinbase 
all_stock_orders.rename(
    columns = {
            "symbol":"Symbol",
            "simple_name":"Name",
            "investment type":"Investment Type",
            "sector":"Sector",
            "industry":"Industry",
            "quantity":"Quantity",
            "executed_notional.amount":"USD Amount"
    },
    inplace = True
)

# Flip the sign on the order share quantity (for all sells) and dollar amount (for all buys)
all_stock_orders["USD Amount"] =  all_stock_orders.apply(
    lambda row: -row["USD Amount"] if row["side"] == "buy" else row["USD Amount"], axis = 1
)
all_stock_orders["Quantity"] =  all_stock_orders.apply(
    lambda row: row["Quantity"] if row["side"] == "buy" else -row["Quantity"], axis = 1
)

# Aggregate all orders into the assets that they were for
all_time_stock_portfolio = all_stock_orders.groupby(
    ["Symbol","Name","Investment Type","Sector","Industry","latest_price"] # this list will be passed to the UDF
).agg({"Quantity":"sum","USD Amount":"sum"})
all_time_stock_portfolio.reset_index(inplace = True)

# Rename cols to reflect a more accurate description 
all_time_stock_portfolio.rename(
    columns = {
        "Quantity":"Current Quantity",
        "USD Amount":"All Time Loss or Gain"
    },
    inplace = True
)

# Set the current equity
all_time_stock_portfolio["latest_price"] = all_time_stock_portfolio["latest_price"] * all_time_stock_portfolio["Current Quantity"]
all_time_stock_portfolio.rename(columns={"latest_price":"Current Equity"}, inplace=True)

# For assets that you currently own, the current equity needs to be added the All Time Net Loss or Gain (it's 
# what your actual loss or gain would be if you were to sell those assets at that equity)
all_time_stock_portfolio["All Time Loss or Gain"] = all_time_stock_portfolio["All Time Loss or Gain"] + all_time_stock_portfolio["Current Equity"]

# Reorder cols
all_time_stock_portfolio = all_time_stock_portfolio[
    ["Symbol","Name","Investment Type","Sector","Industry","Current Quantity","Current Equity","All Time Loss or Gain"]
]

400 Client Error: Bad Request for url: https://api.robinhood.com/fundamentals/?symbols=TGIF


In [5]:
# +++ Coinbase +++
# client = Client(coinbase_key_id, coinbase_key_secret)

# Get all asset accounts (filter out the non-wallet accounts)
crypto_asset_accounts = pd.DataFrame(client.get_accounts()["data"]) # when would i need to use the pagination key? I don't think I have enough accounts, but you will have enought txns...
crypto_asset_accounts = crypto_asset_accounts[crypto_asset_accounts["type"] == "wallet"]

# Add latest price to each asset (using the get exchange rates endpoint)
crypto_asset_accounts['symbol'] = crypto_asset_accounts['balance'].astype(str).str.split(' ').str[0]
crypto_asset_accounts['latest_price'] = crypto_asset_accounts['symbol'].apply( lambda x: float(client.get_exchange_rates(currency = x)['rates']['USD']) )

# Get all txns among those asset accounts
all_crypto_txns = []
for crypto_asset_account_id in crypto_asset_accounts.id.to_list():

    crypto_asset_account_txns = client.get_transactions(crypto_asset_account_id)
    all_crypto_txns += crypto_asset_account_txns.data

    next_starting_after = crypto_asset_account_txns.pagination.next_starting_after
    while next_starting_after is not None:
        crypto_asset_account_txns = client.get_transactions(crypto_asset_account_id, starting_after = next_starting_after)
        all_crypto_txns += crypto_asset_account_txns.data
        next_starting_after = crypto_asset_account_txns.pagination.next_starting_after

# Convert to dataframe
all_crypto_txns = pd.DataFrame(all_crypto_txns)

# Add the asset account id as a new column for each txn
all_crypto_txns['asset_account_id'] = all_crypto_txns['resource_path'].str.split('/').str[3]

# Inner join both dataframes on asset_account_id
all_crypto_txns = all_crypto_txns.merge(crypto_asset_accounts[['id', 'latest_price']], left_on = 'asset_account_id', right_on = 'id', suffixes=('_txn', '_acct'))

del crypto_asset_account_txns

# Filter out this record...
all_crypto_txns = all_crypto_txns[all_crypto_txns["type"] != "staking_transfer"]

# Subset cols
all_crypto_txns = all_crypto_txns[[
    "id_txn",
    "type",
    "amount",
    "native_amount",
    "created_at",
    "latest_price"
]]

# Parse out the USD amount from the native_amount column
all_crypto_txns['native_amount'] = all_crypto_txns.loc[:,'native_amount'].astype(str).str.extract('(-?\d+.\d+)').astype(float)

# Parse out the crypto symbol and quantity from the amount column (drop the original amount column)
all_crypto_txns[['Symbol', 'Quantity']] = all_crypto_txns['amount'].astype(str).str.split(' ', expand=True)
all_crypto_txns['Quantity'] = all_crypto_txns['Quantity'].astype(float)
all_crypto_txns = all_crypto_txns.drop('amount', axis=1)

# Rename the native_amount column to USD Amount
all_crypto_txns.rename(columns={"native_amount": "USD Amount"}, inplace=True)

# Re-catagorize the tx and advanced_trade_fill types as buys
all_crypto_txns.loc[all_crypto_txns['type'] == 'tx', 'type'] = 'buy'
all_crypto_txns.loc[all_crypto_txns['type'] == 'advanced_trade_fill', 'type'] = 'buy'

#  Flip the sign on the sell and buy amounts
all_crypto_txns.loc[all_crypto_txns['type'] == 'sell', 'USD Amount'] = -1 * all_crypto_txns.loc[all_crypto_txns['type'] == 'sell', 'USD Amount']
all_crypto_txns.loc[all_crypto_txns['type'] == 'buy', 'USD Amount'] = -1 * all_crypto_txns.loc[all_crypto_txns['type'] == 'buy', 'USD Amount']

# Aggregate... 
all_time_crypto_portfolio = all_crypto_txns.groupby(
    ["Symbol", "latest_price"] # add latest price at some point 
).agg({"Quantity":"sum","USD Amount":"sum"})
all_time_crypto_portfolio.reset_index(inplace = True)

# Rename columns 
all_time_crypto_portfolio.rename(
    columns={
        "USD Amount": "All Time Loss or Gain",
        "Quantity": "Current Quantity"
    }, 
    inplace=True
)

# Add current equity column
all_time_crypto_portfolio['Current Equity'] = all_time_crypto_portfolio['Current Quantity'] * all_time_crypto_portfolio['latest_price']

# Adjust the all time loss or gain column to reflect the current price
all_time_crypto_portfolio['All Time Loss or Gain'] = all_time_crypto_portfolio['All Time Loss or Gain'] + all_time_crypto_portfolio['Current Equity']

# Round the numbers
all_time_crypto_portfolio['Current Equity'] = all_time_crypto_portfolio['Current Equity'].round(4)
all_time_crypto_portfolio['Current Quantity'] = all_time_crypto_portfolio['Current Quantity'].round(6)

all_time_crypto_portfolio["Name"] = ""
all_time_crypto_portfolio["Industry"] = ""
all_time_crypto_portfolio["Sector"] = ""
all_time_crypto_portfolio["Investment Type"] = "cryptocurrency"

# Reorder columns
all_time_crypto_portfolio = all_time_crypto_portfolio[[
    "Symbol",
    'Name', # wallet name needs to go here.
    'Investment Type', 
    'Sector',
    'Industry',
    "Current Quantity",
    "Current Equity",
    "All Time Loss or Gain"
]]


In [6]:
# concatenate all time stock portfolio with the crypto portfolio
df = pd.concat([all_time_stock_portfolio, all_time_crypto_portfolio])

# # Write holdings data to the workbook and make it a table
# # what's the name of that table again??? 
# holdings_table_address = self.wb.sheets["Personal Investment Portfolio"].tables["holdings"].range.address
# self.wb.sheets["Personal Investment Portfolio"].range( re.sub('\$(\d+)$', increment, holdings_table_address) ).delete(shift = 'up')
# self.wb.sheets["Personal Investment Portfolio"].range("A1").options(index=False).value = df
# self.wb.sheets["Personal Investment Portfolio"].tables.add(source = self.wb.sheets["Personal Investment Portfolio"].range("A1").current_region, name = "holdings")
# self.wb.sheets["Personal Investment Portfolio"].range("A1").current_region.autofit()

# print(df)
# return df